In [3]:
import numpy as np
from PIL import Image, ImageDraw
import tkinter as tk

# Sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivative of sigmoid
def sigmoid_derivative(x):
    return x * (1 - x)

# Loss function: Mean Squared Error
def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Neural Network class
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.weights1 = np.random.randn(input_size, hidden_size) * 0.01
        self.bias1 = np.zeros((1, hidden_size))
        self.weights2 = np.random.randn(hidden_size, output_size) * 0.01
        self.bias2 = np.zeros((1, output_size))

    def forward(self, X):
        self.z1 = np.dot(X, self.weights1) + self.bias1
        self.a1 = sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.weights2) + self.bias2
        self.a2 = sigmoid(self.z2)
        return self.a2

    def backward(self, X, y, learning_rate):
        m = X.shape[0]
        d_a2 = self.a2 - y
        d_z2 = d_a2 * sigmoid_derivative(self.a2)
        d_weights2 = np.dot(self.a1.T, d_z2) / m
        d_bias2 = np.sum(d_z2, axis=0, keepdims=True) / m

        d_a1 = np.dot(d_z2, self.weights2.T)
        d_z1 = d_a1 * sigmoid_derivative(self.a1)
        d_weights1 = np.dot(X.T, d_z1) / m
        d_bias1 = np.sum(d_z1, axis=0, keepdims=True) / m

        self.weights1 -= learning_rate * d_weights1
        self.bias1 -= learning_rate * d_bias1
        self.weights2 -= learning_rate * d_weights2
        self.bias2 -= learning_rate * d_bias2

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = mse_loss(y, y_pred)
            self.backward(X, y, learning_rate)
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss}")

    def predict(self, X):
        return self.forward(X)

# Initialize global variables
canvas_size = (100, 100)  # Canvas dimensions
draw_color = 0  # Black
background_color = 255  # White
image = Image.new("L", canvas_size, background_color)
draw = ImageDraw.Draw(image)
data_collected = []

# Function to draw on the canvas
def draw_on_canvas(event):
    x, y = event.x, event.y
    draw.ellipse([x, y, x + 5, y + 5], fill=draw_color)
    canvas.create_oval(x, y, x + 5, y + 5, fill="black")

# Function to save the drawn image
def save_image():
    global data_collected
    img_resized = image.resize(canvas_size)
    binary_image = np.array(img_resized).flatten() / 255.0  # Normalize to [0, 1]
    label = int(input("Enter the label for the drawn character (e.g., 0-25 for A-Z): "))
    data_collected.append((binary_image, label))
    print("Image saved with label:", label)

# Function to train the neural network
def train_model():
    if len(data_collected) == 0:
        print("No data collected yet!")
        return

    X = np.array([x[0] for x in data_collected])
    y = np.array([x[1] for x in data_collected]).reshape(-1, 1)
    y_one_hot = np.zeros((y.size, 26))  # One-hot encode labels for 26 letters
    y_one_hot[np.arange(y.size), y.flatten()] = 1

    nn = NeuralNetwork(input_size=10000, hidden_size=64, output_size=26)
    nn.train(X, y_one_hot, epochs=1000, learning_rate=0.1)
    print("Training complete!")

# Function to clear the canvas
def clear_canvas():
    global image, draw
    canvas.delete("all")
    image = Image.new("L", canvas_size, background_color)
    draw = ImageDraw.Draw(image)

# Create the Tkinter window
root = tk.Tk()
root.title("Handwriting Pad")

# Create a canvas for drawing
canvas = tk.Canvas(root, width=canvas_size[0], height=canvas_size[1], bg="white")
canvas.pack()
canvas.bind("<B1-Motion>", draw_on_canvas)

# Buttons for saving and training
btn_save = tk.Button(root, text="Save Image", command=save_image)
btn_save.pack()

btn_train = tk.Button(root, text="Train Model", command=train_model)
btn_train.pack()

btn_clear = tk.Button(root, text="Clear Canvas", command=clear_canvas)
btn_clear.pack()

root.mainloop()


TclError: no display name and no $DISPLAY environment variable